[View in Colaboratory](https://colab.research.google.com/github/maddy46/mlblr4a_experiments/blob/master/DNST_CIFAR10_AUG_final.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /content/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
  Running setup.py bdist_wheel for humanize ... - done
  Stored in directory: /content/.cache/pip/wheels/69/86/6c/f8b8593bc273ec4b0c653d3827f7482bb2001a2781a73b7f44
Successfully built humanize
Gen RAM Free: 12.4 GB  I Proc size: 668.5 MB
GPU RAM Free: 11330MB | Used: 109MB | Util   1% | Total 11439MB


In [5]:
!rm utils.py
!wget https://raw.githubusercontent.com/Zahlii/colab-tf-utils/master/utils.py
import utils
import os
import keras

def compare(best, new):
  return best.losses['val_acc'] < new.losses['val_acc']

def path(new):
  if new.losses['val_acc'] > 0.2:
    return '4B_008_%s.h5' % new.losses['val_acc']


  
saver = utils.GDriveCheckpointer(compare,path)

In [6]:
# Hyperparameters
num_classes = 10
epochs =50
num_filter = 64
compression = 0.5
dropout_rate = 0.25
l = 6
batch_size = 32

In [7]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print (img_height)
print (img_width)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 28s 0us/step
32
32


In [8]:
print (x_train.shape[0])
print (x_test.shape[0])

50000
10000


In [9]:
'''
new_images = np.empty((50000, 32, 32, 3), dtype='uint8')
new_test = np.empty((10000, 32, 32, 3), dtype='uint8')

for image_count in range(x_train.shape[0]):
    new_images[image_count] = cv2.resize(x_train[image_count], (32, 32))
for image_count1 in range(x_test.shape[0]):
    new_test[image_count1] = cv2.resize(x_test[image_count1], (32, 32))
img_height1, img_width1, channel1 = new_images.shape[1],new_images.shape[2],new_images.shape[3]
print (img_height1)
print (img_width1)

print('x resizing Done.')
'''

"\nnew_images = np.empty((50000, 32, 32, 3), dtype='uint8')\nnew_test = np.empty((10000, 32, 32, 3), dtype='uint8')\n\nfor image_count in range(x_train.shape[0]):\n    new_images[image_count] = cv2.resize(x_train[image_count], (32, 32))\nfor image_count1 in range(x_test.shape[0]):\n    new_test[image_count1] = cv2.resize(x_test[image_count1], (32, 32))\nimg_height1, img_width1, channel1 = new_images.shape[1],new_images.shape[2],new_images.shape[3]\nprint (img_height1)\nprint (img_width1)\n\nprint('x resizing Done.')\n"

In [10]:
datagen = ImageDataGenerator(zoom_range=0.2, rotation_range=20,width_shift_range=0.2,height_shift_range=0.2)
datagen.fit(x_train)

In [11]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [12]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [13]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [14]:

input = Input(shape=(img_height, img_width, channel,))
print(channel)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)
#output = output_layer(First_Block)


3


In [15]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [16]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [18]:
'''
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=cb)
                    '''

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, 
                    verbose=1, validation_data=(x_test
                                                , y_test),
                    steps_per_epoch=len(x_train)/batch_size, callbacks=[saver])

Epoch 1/50
1562/1562 [============================>.] - ETA: 0s - loss: 1.6509 - acc: 0.3957

1563/1562 [==============================] - 298s 191ms/step - loss: 1.6507 - acc: 0.3958 - val_loss: 2.1890 - val_acc: 0.3780


Uploading file 4B_008_0.378.h5 to folder Colab Notebooks: 100%|██████████| 100/100 [00:00<00:00, 105.63it/s]


Epoch 2/50
1562/1562 [============================>.] - ETA: 0s - loss: 1.2897 - acc: 0.5359

1563/1562 [==============================] - 292s 187ms/step - loss: 1.2894 - acc: 0.5360 - val_loss: 1.9765 - val_acc: 0.4965
Removing old cloud file 4B_008_0.378.h5


...

1563/1562 [==============================] - 289s 185ms/step - loss: 0.3735 - acc: 0.8697 - val_loss: 0.5332 - val_acc: 0.8557
No improvement.
Epoch 49/50
 419/1562 [=======>......................] - ETA: 3:20 - loss: 0.3555 - acc: 0.8770

1563/1562 [==============================] - 290s 185ms/step - loss: 0.3736 - acc: 0.8703 - val_loss: 0.4520 - val_acc: 0.8740
Removing old cloud file 4B_008_0.8685.h5


Uploading file 4B_008_0.874.h5 to folder Colab Notebooks: 100%|██████████| 100/100 [00:00<00:00, 116.64it/s]


Epoch 50/50
1562/1562 [============================>.] - ETA: 0s - loss: 0.3677 - acc: 0.8716

1563/1562 [==============================] - 291s 186ms/step - loss: 0.3677 - acc: 0.8716 - val_loss: 0.4548 - val_acc: 0.8699
No improvement.


In [19]:
# Save the trained weights in to .h5 format
model.save_weights("saved_model.h5")
print("Saved model to disk")

Saved model to disk
